<a href="https://colab.research.google.com/github/rexbrandy/Neural_Networks/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install 'portalocker>=2.0.0'

In [8]:
!python -m spacy download en_core_web_sm

2024-01-19 14:17:15.251163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 14:17:15.251248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 14:17:15.253618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 14:17:15.264812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 14:17:16.318737: W tensorflow/compiler/tf2

In [21]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

SEED = 1111

train_iter = iter(AG_NEWS(split=('train')))
tokenizer = get_tokenizer('basic_english')

def yield_tokens(iter):
    for _, text in iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

# The text pipeline converts a text string into a list of integers based on the lookup table defined in the vocabulary.
# The label pipeline converts the label into integers
#
# print(text_pipeline('here is the an example'), label_pipeline('10')) >> [475, 21, 2, 30, 5297], 9
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

print(text_pipeline('here is the an example'), label_pipeline('10'))

[475, 21, 2, 30, 5297] 9


In [ ]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    for _label, _text in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)

        text_list.append(processed_text)
        label_list.append(label_pipeline(_label))
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[: -1]).cumsum(dim=0)
    text_list = torch.cat(text_list)



In [ ]:
from torch import nn

### The Model
class TextClassificationModel(nn.Model):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparce=False)
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [ ]:
import time

def train(model, dataloader, optim, criterion, epoch):
    model.train()
    total_accuracy, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optim.zero_grad()

        prediction = model(text, offsets)

        loss = criterion(prediction, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optim.step()

        total_accuracy += (prediction.argmax(1) == 1).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f'Epoch: {epoch} | Total {idx}/{len(dataloader)} | Accuracy {total_acc / total_count}')

            total_acc, total_count = 0, 0
            start_time = time.time()
